In [1]:
import tensorflow as tf
import numpy as np
tf.set_random_seed(777)  # reproducibility


In [2]:
sample = " if you want you"
idx2char = list(set(sample))  # index -> char
char2idx = {c: i for i, c in enumerate(idx2char)}  # char -> idex


In [5]:
print(idx2char)
print(char2idx)

['y', ' ', 'w', 'f', 'a', 'n', 'u', 't', 'o', 'i']
{'y': 0, ' ': 1, 'w': 2, 'f': 3, 'a': 4, 'n': 5, 'u': 6, 't': 7, 'o': 8, 'i': 9}


In [6]:
sample_idx = [char2idx[c] for c in sample]  # char to index
x_data = [sample_idx[:-1]]  # X data sample (0 ~ n-1) hello: hell
y_data = [sample_idx[1:]]   # Y label sample (1 ~ n) hello: ello

In [8]:
x_data

[[1, 9, 3, 1, 0, 8, 6, 1, 2, 4, 5, 7, 1, 0, 8]]

In [9]:
y_data

[[9, 3, 1, 0, 8, 6, 1, 2, 4, 5, 7, 1, 0, 8, 6]]

In [14]:
# hyper parameters
dic_size = len(char2idx)  # RNN input size (one hot size)
hidden_size = len(char2idx)  # RNN output size
num_classes = len(char2idx)  # final output size (RNN or softmax, etc.)
batch_size = 1  # one sample data, one batch
sequence_length = len(sample) - 1  # number of lstm rollings (unit #)
learning_rate = 0.1

X = tf.placeholder(tf.int32, [None, sequence_length])  # X data
Y = tf.placeholder(tf.int32, [None, sequence_length])  # Y label

x_one_hot = tf.one_hot(X, num_classes)  # one hot: 1 -> 0 1 0 0 0 0 0 0 0 0

In [15]:
x_one_hot

<tf.Tensor 'one_hot_1:0' shape=(?, 15, 10) dtype=float32>

In [16]:
cell = tf.contrib.rnn.BasicLSTMCell(
    num_units=hidden_size, state_is_tuple=True)
initial_state = cell.zero_state(batch_size, tf.float32)
outputs, _states = tf.nn.dynamic_rnn(
    cell, x_one_hot, initial_state=initial_state, dtype=tf.float32)

# FC layer
X_for_fc = tf.reshape(outputs, [-1, hidden_size])
outputs = tf.contrib.layers.fully_connected(X_for_fc, num_classes, activation_fn=None)

# reshape out for sequence_loss
outputs = tf.reshape(outputs, [batch_size, sequence_length, num_classes])

weights = tf.ones([batch_size, sequence_length])
sequence_loss = tf.contrib.seq2seq.sequence_loss(
    logits=outputs, targets=Y, weights=weights)
loss = tf.reduce_mean(sequence_loss)
train = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)

prediction = tf.argmax(outputs, axis=2)

In [17]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(50):
        l, _ = sess.run([loss, train], feed_dict={X: x_data, Y: y_data})
        result = sess.run(prediction, feed_dict={X: x_data})

        # print char using dic
        result_str = [idx2char[c] for c in np.squeeze(result)]

        print(i, "loss:", l, "Prediction:", ''.join(result_str))

0 loss: 2.30188 Prediction: yyyyoooooooyooo
1 loss: 2.16103 Prediction:      uuuuuuuouu
2 loss: 2.07389 Prediction:      u        u
3 loss: 1.9379 Prediction:      u        u
4 loss: 1.80821 Prediction:      u       ou
5 loss: 1.58889 Prediction:    youuoun  you
6 loss: 1.33929 Prediction:    youuwunt you
7 loss: 1.08475 Prediction: i  you want you
8 loss: 0.814583 Prediction: if you want you
9 loss: 0.604403 Prediction: if you want you
10 loss: 0.438769 Prediction: if you want you
11 loss: 0.311746 Prediction: if you want you
12 loss: 0.221046 Prediction: if you want you
13 loss: 0.155526 Prediction: if you want you
14 loss: 0.108418 Prediction: if you want you
15 loss: 0.0756977 Prediction: if you want you
16 loss: 0.0533878 Prediction: if you want you
17 loss: 0.0381673 Prediction: if you want you
18 loss: 0.0277041 Prediction: if you want you
19 loss: 0.0204334 Prediction: if you want you
20 loss: 0.0153127 Prediction: if you want you
21 loss: 0.0116544 Prediction: if you want you
